# MySQL PRNG Analysis

While analyzing MySQL's PRNG, I discovered some interesting analysis techniques.
The goals are to:
1. Solve the internal state given only outputs
2. Run the PRNG in reverse to determine past outputs.

Here's the code for MySQL's PRNG:

https://github.com/mysql/mysql-server/blob/3e90d07c3578e4da39dc1bce73559bbdf655c28c/sql/auth/password.cc#L81
```c++
void randominit(struct rand_struct *rand_st, ulong seed1,
                ulong seed2) { /* For mysql 3.21.# */
  rand_st->max_value = 0x3FFFFFFFL;
  rand_st->max_value_dbl = (double)rand_st->max_value;
  rand_st->seed1 = seed1 % rand_st->max_value;
  rand_st->seed2 = seed2 % rand_st->max_value;
}
```

https://github.com/mysql/mysql-server/blob/3e90d07c3578e4da39dc1bce73559bbdf655c28c/mysys/my_rnd.cc#L50
```c++
double my_rnd(struct rand_struct *rand_st) {
  rand_st->seed1 = (rand_st->seed1 * 3 + rand_st->seed2) % rand_st->max_value;
  rand_st->seed2 = (rand_st->seed1 + rand_st->seed2 + 33) % rand_st->max_value;
  return (((double)rand_st->seed1) / rand_st->max_value_dbl);
}
```

To simplify this, let's set change some variables and write this in Python. Here's the mapping:
```
a0 = rand_st->seed1
b0 = rand_st->seed2
m = rand_st->max_value = 0x3FFFFFFF
```

```python
def my_rnd():
  a1 = (a0 * 3 + b0) % m
  b1 = (a1 + b0 + 33) % m
  return a1
```

This looks like some sort of two-state LCG. Generally, LCGs are of the form `X2 = (X1 * c + d) (mod m)`, and we can trivially solve them algebraically. If you have state `X2` and want state `X1`, you just rearrange the equation to be `X1 = (X2 - d)/c (mod m)`. Here, division by `c` is done via taking its multiplicative inverse.

However, MySQL's PRNG has a few interesting properties:
1. The increment, `b`, also changes
2. The multiplier, 3, is _not_ invertible over the modulus

I don't know if the code authors wrote it wrong, but the way the modulus is being used is non-standard. Typically, you would `&` with the modulus if it's of the form 2^n-1 which makes it equivalent to `mod 2^n`. Their use of modulo means the modulus is 2^30-1 instead of 2^30. The modulus factors to `{3: 2, 7: 1, 11: 1, 31: 1, 151: 1, 331: 1}`. Since it's not coprime to 3, 3 cannot be inverted. Something you'll notice is that the constants 3 and 33 are both divisible by 3. If we track the sequences of congruence mod 3 for `a` and `b`, we can make some simple observations.

```python
def my_rnd():
  a1 = (a0 * 3 + b0) % m  # a1 = b0 (mod 3)
  b1 = (a1 + b0 + 33) % m # b1 = 2*b0 (mod 3)
  return a1
```

Let's attempt to make a simple reverse function by rearranging the equations.

```python
def rev(a1, b1, m):
    b0 = (b1 - a1 - 33) % m
    a0 = ((a1 - b0)//3) % m # b0 = a1 (mod 3), so (a1 - b0) is divisible by 3
    return a0, b0
```

In [1]:
from samson.all import *

# Initialize variables
m = 2**30-1
a0 = 1036040342
b0 = 158423163

print("a0", a0)
print("b0", b0)

def my_rng(a, b, m):
    a1 = (a*3 + b) % m
    b1 = (a1 + b + 33) % m
    return a1, b1

def rev(a1, b1, m):
    b0 = (b1 - a1 - 33) % m
    a0 = ((a1 - b0)//3) % m
    return a0, b0

a1, b1 = my_rng(a0, b0, m)
print('It works! Well, so far...')
print(rev(a1, b1, m))

def test_reverse(a0, b0, rev_func):
    a1, b1 = a0, b0
    outputs = []

    for _ in range(10):
        a1, b1 = my_rng(a1, b1, m)
        outputs.append((a1, b1))

    rev_out = [(a1, b1)]
    for _ in range(9):
        a1, b1 = rev_func(a1, b1, m)
        rev_out.append((a1, b1))
    
    return outputs == rev_out[::-1]

print('Does it hold up for multiple inputs?', test_reverse(a0, b0, rev))

a0 1036040342
b0 158423163
It works! Well, so far...
(1036040342, 158423163)
Does it hold up for multiple inputs? False


What's going on here? I thought the reverse function worked! The problem is that we found one of the possible previous states and not _the_ previous state. Check it out:

In [10]:
print(my_rng(a0, b0, m))
print(my_rng(320212460, b0, m))
print(my_rng(678126401, b0, m))

(45318720, 203741916)
(45318720, 203741916)
(45318720, 203741916)


However, it's very important that we get the _right_ state. Even if we found a way to generate all three possible states, if the end user wanted the 10th output back, they'd have 3^10 (59049) possible outputs. Okay, so what if we could 1) generate all three possible previous states and then 2) figure out which one is correct? Let's do some more analysis.

First, we need to find the root cause, which comes back to the inversion problem. When you can invert the multiplier, there's only one solution.

In [8]:
a1, b1 = my_rng(a0, b0, m)
print(a1, b1)
z = (a1-b0)
y = z // 3
m9 = m // 9
res = [r[0] for r in [crt([(i, 9), (y % m9, m9)]) for i in range(9)]]
print(res)

45318720 203741916
[558821754, 439517107, 320212460, 200907813, 81603166, 1036040342, 916735695, 797431048, 678126401]
